In [1]:
###el bueno
import scipy.io
import numpy

import torchtext
import torch
from torch.autograd import Variable
import torch.nn as nn
from torchtext.vocab import Vectors, GloVe
import torch.nn.functional as F
import pdb
from copy import deepcopy
import pdb

In [18]:
##Graph a
##family1:
num_constants = 6
#Background
edge_extension = torch.zeros(num_constants, num_constants)
edge_extension[0,1] = 1
edge_extension[1,2] = 1
edge_extension[1,3] = 1
edge_extension[2,0] = 1
edge_extension[3,4] = 1
edge_extension[3,5] = 1
edge_extension[4,5] = 1
edge_extension[5,4] = 1

#Intensional Predicates
aux_extension = torch.zeros(num_constants,num_constants)
target_extension = torch.zeros(1,num_constants)

valuation_init = [Variable(edge_extension), Variable(aux_extension), Variable(target_extension)]

num_predicates= len(valuation_init)
intensional_predicates=[1,2]
num_intensional_predicates = len(intensional_predicates)

#Target
target = torch.zeros(num_constants,num_constants)
target[0] = torch.zeros(1,num_constants)
target_aux = [0,1,2,4,5]
for integer in target_aux:
    target[0,integer]=1
    
valuation_inits = []
targets = []
valuation_inits.append(deepcopy(valuation_init))   
targets.append(target)

In [19]:
##family1:
num_constants = 5
#Background
edge_extension = torch.zeros(num_constants, num_constants)
edge_extension[0,1] = 1
edge_extension[2,3] = 1
edge_extension[3,4] = 1
edge_extension[4,2] = 1


#Intensional Predicates
aux_extension = torch.zeros(num_constants,num_constants)
target_extension = torch.zeros(1,num_constants)

valuation_init = [Variable(edge_extension), Variable(aux_extension), Variable(target_extension)]

num_predicates= len(valuation_init)
intensional_predicates=[1,2]
num_intensional_predicates = len(intensional_predicates)

#Target
target = torch.zeros(num_constants,num_constants)
target[0] = torch.zeros(1,num_constants)
target_aux = [2,3,4]
for integer in target_aux:
    target[0,integer]=1
    
valuation_inits.append(deepcopy(valuation_init))   
targets.append(target)

In [6]:
print(valuation_init[0], valuation_init[2], target, valuation_inits[0])

(Variable containing:
 0  1  0  0  0
 0  0  0  0  0
 0  0  0  1  0
 0  0  0  0  1
 0  0  1  0  0
[torch.FloatTensor of size 5x5]
, Variable containing:
 0  0  0  0  0
[torch.FloatTensor of size 1x5]
, 
 0  0  1  1  1
 0  0  0  0  0
 0  0  0  0  0
 0  0  0  0  0
 0  0  0  0  0
[torch.FloatTensor of size 5x5]
, [Variable containing:
 0  1  0  0  0  0
 0  0  1  1  0  0
 1  0  0  0  0  0
 0  0  0  0  1  1
 0  0  0  0  0  1
 0  0  0  0  1  0
[torch.FloatTensor of size 6x6]
, Variable containing:
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
[torch.FloatTensor of size 6x6]
, Variable containing:
 0  0  0  0  0  0
[torch.FloatTensor of size 1x6]
])


In [16]:
def decoder_efficient(valuation, step, num_constants):
    
    ## Create valuation_new
    valuation_new = [deepcopy(valuation[0]),Variable(torch.zeros(valuation[1].size())),Variable(torch.zeros(valuation[2].size()))]
                    
    ##Unifications
    rules_aux = torch.cat((rules[:,:num_feat],rules[:,num_feat:2*num_feat],rules[:,2*num_feat:3*num_feat]),0)
    rules_aux = rules_aux.repeat(num_predicates,1)
    embeddings_aux = embeddings.repeat(1,num_rules*3).view(-1,num_feat)
    #embeddings_aux = F.dropout(embeddings_aux, p=.1,training=True)
    #rules_aux = F.dropout(rules_aux, p=.1, training=True)
    
    #unifs = F.cosine_similarity(embeddings_aux, rules_aux).view(num_predicates,-1)
    
    unifs = F.pairwise_distance(embeddings_aux, rules_aux).view(num_predicates,-1)
    unifs = torch.exp(-unifs)
    unifs_sum = torch.sum(unifs, 0)
    unifs= unifs/unifs_sum
    
    
    ###TODO I want to normalize across the rules!!
    #unifs = rbf(embeddings_aux, rules_aux).view(num_predicates,-1)
    #unifs = F.relu(unifs)
    #unifs_sum = torch.sum(unifs, 0)
    #unifs= unifs/unifs_sum
    #unifs = torch.min(unifs,Variable(torch.ones(unifs.size())))

    ##Get_Valuations
    for predicate in intensional_predicates:
        if valuation[predicate].size()[0] == 1:
            #pdb.set_trace()
            for s in range(num_constants):
                valuation_aux = Variable(torch.Tensor([0]))
                for body1 in range(num_predicates):
                    for body2 in range(num_predicates):
                        ## Get nums
                        if valuation[body1].size()[0] == 1:
                            if valuation[body2].size()[0] == 1:
                                num = torch.min(valuation[body1][0,s],valuation[body2][0,s])
                            else: 
                                num = torch.min(valuation[body1][0,:],valuation[body2][:,s])
                                num = torch.max(num)
                        else:
                            if valuation[body2].size()[0] == 1:
                                num = torch.min(valuation[body1][:,s],valuation[body2][0,s])
                                num = torch.max(num)
                            else:
                                num = torch.min(valuation[body1][s,:],valuation[body2][:,s])
                                #num = torch.min(valuation[body1][s,:],valuation[body2][:,o])
                                num = torch.max(num)

                        ## max across three rules
                        new = Variable(torch.Tensor([0]))
                        for rule in range(num_rules): 
                            unif = unifs[predicate][rule]*unifs[body1][num_rules+rule]*unifs[body2][2*num_rules+rule]
                            new = torch.max(new,unif) 

                        num = num*new 
                        valuation_aux = torch.max(valuation_aux, num)
                        if valuation_aux.data[0] > 1:
                                pdb.set_trace()
                        if predicate == 3 and s==0 and num.data[0]>.3:
                            print('bodyyyyy0',body1, body2)
                        if predicate == 3 and s==2 and num.data[0]>.3:
                            print('bodyyyyy2',body1, body2)
                        if predicate == 3 and s==4 and num.data[0]>.3:
                            print('bodyyyyy4',body1, body2) 
                        #if predicate == 3 and s==0 and o==6 and num.data[0]>.3:
                            #print('bodyyyyy6',body1, body2)
                        #if predicate == 3 and s==0 and o==8 and num.data[0]>.3:
                            #print('bodyyyyy8',body1, body2)   
                valuation_new[predicate][0,s] = torch.max(valuation[predicate][0,s], valuation_aux) 
            
            
            
        else:
            for s in range(num_constants):
                for o in range(num_constants):
                    valuation_aux = Variable(torch.Tensor([0]))
                    for body1 in range(num_predicates):
                        for body2 in range(num_predicates):
                            ## Get nums
                            if valuation[body1].size()[0] == 1:
                                if valuation[body2].size()[0] == 1:
                                    num = torch.min(valuation[body1][0,s],valuation[body2][0,o])
                                else: 
                                    num = torch.min(valuation[body1][0,s],valuation[body2][s,o])
                                    #num = torch.max(num)
                            else:
                                if valuation[body2].size()[0] == 1:
                                    num = torch.min(valuation[body1][s,o],valuation[body2][0,o])
                                    #num = torch.max(num)
                                else: 
                                    num = torch.min(valuation[body1][s,:],valuation[body2][:,o])
                                    num = torch.max(num)
                                    num_aux = torch.min(valuation[body1][s,o],valuation[body2][s,o])
                                    num = torch.max(num, num_aux)

                            ## max across three rules
                            new = Variable(torch.Tensor([0]))
                            for rule in range(num_rules): 
                                unif = unifs[predicate][rule]*unifs[body1][num_rules+rule]*unifs[body2][2*num_rules+rule]
                                new = torch.max(new,unif)
                                #could be amalgamate

                            num = num*new 
                            valuation_aux = torch.max(valuation_aux, num)
                            if valuation_aux.data[0] > 1:
                                pdb.set_trace()
                            if predicate == 3 and s==0 and o==1 and num.data[0]>.3:
                                print('bodyyyyy1',body1, body2)
                            if predicate == 3 and s==0 and o==3 and num.data[0]>.3:
                                print('bodyyyyy3',body1, body2)
                            if predicate == 3 and s==0 and o==5 and num.data[0]>.3:
                                print('bodyyyyy5',body1, body2) 
                            #if predicate == 3 and s==0 and o==6 and num.data[0]>.3:
                                #print('bodyyyyy6',body1, body2)
                            #if predicate == 3 and s==0 and o==8 and num.data[0]>.3:
                                #print('bodyyyyy8',body1, body2)     
                    valuation_new[predicate][s,o] = torch.max(valuation[predicate][s,o], valuation_aux) 
                
    return valuation_new
            
def amalgamate(x,y):
    return x + y - x*y

def rbf(x,y):
    ans = (x-y)**2
    ans = torch.mean(ans,1)
    ans = torch.exp(-ans)
    return ans



In [31]:
num_iters = 50
learning_rate = .1
steps = 3

num_feat=3
num_rules = 3


embeddings = Variable(torch.rand(num_predicates, num_feat), requires_grad=True)
#rules = Variable(torch.rand(num_rules, num_feat*3), requires_grad=True)
#optimizer = torch.optim.Adam([embeddings,rules],lr=learning_rate)

#embeddings = Variable(torch.eye(num_predicates), requires_grad=True)
rules = Variable(torch.rand(num_rules, num_feat*3), requires_grad=True)
#rule1 = torch.Tensor([0,0,1,0,1,0,0,1,0]).view(1,-1)
#rule2 = torch.Tensor([0,1,0,1,0,0,0,1,0]).view(1,-1)
#rule3 = torch.Tensor([0,1,0,0,1,0,0,1,0]).view(1,-1)
#rules = Variable(torch.cat((rule1,rule2,rule3),0), requires_grad=True)

optimizer = torch.optim.Adam([rules, embeddings],lr=learning_rate)

criterion = torch.nn.BCELoss(size_average=False)



#####TOGETHER
for i in range(num_iters):
    total_loss = Variable(torch.Tensor([0]))
    for j in range(len(valuation_inits)):
        optimizer.zero_grad()

        valuation = valuation_inits[j]
        for step in range(steps):
            valuation = decoder_efficient(valuation,step,valuation[0].size()[1])
            print('step',step,'valuation2', valuation[2], 'valuation1',valuation[1]) 
        loss = criterion(valuation[2][0,:],Variable(torch.Tensor(targets[j][0,:])))
        total_loss += loss 
    print(i,'lossssssssssssssssssssssssssss',total_loss.data[0]/2)
    loss.backward()
    optimizer.step()




###SEPARATE
#for i in range(num_iters):
#    optimizer.zero_grad()
#    
#    valuation = valuation_inits[i%2]
#    for step in range(steps):
#        valuation = decoder_efficient(valuation, step, valuation[0].size()[1])
#        print('step',step,'valuation2', valuation[2], 'valuation1',valuation[1]) 
#    loss = criterion(valuation[2][0,:],Variable(torch.Tensor(targets[i%2][0,:])))
    
#    print(i,'lossssssssssssssssssssssssssss',loss.data[0])
#    loss.backward()
#    optimizer.step()
    
    

('step', 0, 'valuation2', Variable containing:
1.00000e-02 *
  0.0000  0.0000  0.0000  0.0000  5.3660  5.3660
[torch.FloatTensor of size 1x6]
, 'valuation1', Variable containing:
1.00000e-02 *
  0.0000  6.6994  6.6994  6.6994  0.0000  0.0000
  6.6994  0.0000  6.6994  6.6994  6.6994  6.6994
  6.6994  6.6994  0.0000  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  0.0000  6.6994  6.6994
  0.0000  0.0000  0.0000  0.0000  6.6994  6.6994
  0.0000  0.0000  0.0000  0.0000  6.6994  6.6994
[torch.FloatTensor of size 6x6]
)
('step', 1, 'valuation2', Variable containing:
1.00000e-02 *
  0.3948  0.3948  0.3948  0.0000  5.3660  5.3660
[torch.FloatTensor of size 1x6]
, 'valuation1', Variable containing:
1.00000e-02 *
  0.3767  6.6994  6.6994  6.6994  0.3767  0.3767
  6.6994  0.3767  6.6994  6.6994  6.6994  6.6994
  6.6994  6.6994  0.3767  0.3767  0.2462  0.2462
  0.0000  0.0000  0.0000  0.0000  6.6994  6.6994
  0.0000  0.0000  0.0000  0.0000  6.6994  6.6994
  0.0000  0.0000  0.0000  0.0000  6.6994 

KeyboardInterrupt: 

In [21]:
print(embeddings)
print(rules)

Variable containing:
-1.4370 -0.1867  1.8656
 2.6922  0.3773  0.3658
 2.9018  3.4198 -1.4301
[torch.FloatTensor of size 3x3]

Variable containing:
 3.2058  1.8425 -0.8803  0.9029 -1.1350  1.9732 -1.4594 -0.1837  1.8494
 0.2143  0.7905  0.4983  0.9086  0.3549  0.3972  0.8405  0.7036  0.5364
 0.5579  0.2857  0.4213  0.9388  0.4618  0.3627  0.5307  0.2135  0.4208
[torch.FloatTensor of size 3x9]



In [30]:
#print(F.cosine_similarity(torch.randn(1,4),torch.randn(1,4)))
#print(embeddings)
#print(rules)
#for embedding in embeddings:
    #print(embedding,rules[0,:4])
    #print(F.cosine_similarity(embedding.view(1,-1),rules[0,4:8].view(1,-1)))
#print(embeddings)
#print(rules)
rules_aux = torch.cat((rules[:,:num_feat],rules[:,num_feat:2*num_feat],rules[:,2*num_feat:3*num_feat]),0)
rules_aux = rules_aux.repeat(num_predicates,1)
embeddings_aux = embeddings.repeat(1,num_rules*3).view(-1,num_feat)
#embeddings_aux = F.dropout(embeddings_aux, p=.1,training=True)
#rules_aux = F.dropout(rules_aux, p=.1, training=True)    
unifs = F.cosine_similarity(embeddings_aux, rules_aux).view(num_predicates,-1)
#print('aaaaa',unifs)
#print('oooo',F.pairwise_distance(embeddings_aux, rules_aux).view(num_predicates,-1))

unifs = F.pairwise_distance(embeddings_aux, rules_aux).view(num_predicates,-1)
unifs = torch.exp(-unifs)
unifs_sum = torch.sum(unifs, 0)
unifs= unifs/unifs_sum
print('finaaaal',unifs)


('finaaaal', Variable containing:
 0.0113  0.6327  0.5051  0.5010  0.4053  0.2548  0.9913  0.4603  0.2525
 0.4936  0.3337  0.4701  0.4890  0.5592  0.6902  0.0080  0.4913  0.7093
 0.4951  0.0335  0.0247  0.0100  0.0355  0.0550  0.0007  0.0484  0.0382
[torch.FloatTensor of size 3x9]
)


In [ ]:

input =torch.randn(5,7)
print(input)
print(F.dropout(input, p=.2,training=True))